In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Description

The goal of this notebook is to demonstrate the various approaches of working with `dataflow`.

# Imports

In [4]:
import logging

import pandas as pd

import core.config.config_ as cconconf
import core.finance as cofinanc
import core.finance.resampling as cfinresa
import core.finance.returns as cfinretu
import dataflow.core as dtfcore
import dataflow.core.utils as dtfcorutil
import dataflow.universe as dtfuniver
import helpers.hdbg as hdbg
import helpers.hprint as hprint
import im_v2.ccxt.data.client as icdcl
import market_data as mdata

In [5]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-93312f32-ece6-4c59-a17d-686e4437997d.json'


# Config

In [6]:
def get_gallery_dataflow_example_config() -> cconconf.Config:
    """
    Get config, that specifies params for getting raw data.
    """
    config = cconconf.Config()
    # Load parameters.
    config.add_subconfig("load")
    config["load"]["data_snapshot"] = "latest"
    config["load"]["dataset"] = "ohlcv"
    config["load"]["contract_type"] = "futures"
    config["load"]["universe_version"] = "v7"
    config["load"]["data_version"] = "v2"
    # Data parameters.
    config.add_subconfig("data")
    config["data"]["start_date"] = pd.Timestamp("2021-09-01", tz="UTC")
    config["data"]["end_date"] = pd.Timestamp("2021-09-15", tz="UTC")
    config["data"]["resampling_rule"] = "5T"
    return config

In [7]:
config = get_gallery_dataflow_example_config()
print(config)

load: 
  data_snapshot: latest
  dataset: ohlcv
  contract_type: futures
  universe_version: v7
  data_version: v2
data: 
  start_date: 2021-09-01 00:00:00+00:00
  end_date: 2021-09-15 00:00:00+00:00
  resampling_rule: 5T


# Load historical data

## Get IM client

In [8]:
data_version = config["load"]["data_version"]
universe_version = config["load"]["universe_version"]
contract_type = config["load"]["contract_type"]
dataset = config["load"]["dataset"]
data_snapshot = config["load"]["data_snapshot"]
client = icdcl.ccxt_clients_example.get_CcxtHistoricalPqByTileClient_example1(
    data_version, universe_version, dataset, contract_type, data_snapshot
)

## Get universe

In [9]:
# Set the universe.
universe_str = "ccxt_v7-all"
full_symbols = dtfuniver.get_universe(universe_str)
asset_ids = client.get_asset_ids_from_full_symbols(full_symbols)

## Get market data loader

In [10]:
columns = None
columns_remap = None
wall_clock_time = pd.Timestamp("2100-01-01T00:00:00+00:00")
market_data = mdata.market_data_example.get_HistoricalImClientMarketData_example1(
    client, asset_ids, columns, columns_remap, wall_clock_time=wall_clock_time
)

## Get data

In [11]:
start_ts = config["data"]["start_date"]
end_ts = config["data"]["end_date"]
ts_col_name = "timestamp"
data_hist = market_data.get_data_for_interval(
    start_ts, end_ts, ts_col_name, asset_ids
)
print(data_hist.shape)
data_hist.head(3)

WARNING * Failed assertion *
val1 - val2=['binance::APE_USDT', 'binance::GMT_USDT']
val2 - val1=[]
val1=27 [binance::APE_USDT, ... binance::XRP_USDT]
set eq
val2=25 [binance::AVAX_USDT, ... binance::XRP_USDT]
Not all the requested symbols were retrieved
Continuing as per user request with only_warning=True
(490829, 9)


,asset_id,full_symbol,open,high,low,close,volume,knowledge_timestamp,start_ts
end_ts,,,,,,,,,
2021-08-31 20:00:00-04:00,1182743717,binance::BTC_BUSD,47192.70,47205.70,47171.20,47171.20,13.499,2022-07-09 16:21:44.328375+00:00,2021-08-31 19:59:00-04:00
2021-08-31 20:00:00-04:00,1464553467,binance::ETH_USDT,3432.96,3433.78,3430.13,3430.49,944.407,2022-06-24 11:10:10.287766+00:00,2021-08-31 19:59:00-04:00
2021-08-31 20:00:00-04:00,1467591036,binance::BTC_USDT,47184.11,47186.11,47147.01,47150.32,150.469,2022-06-24 05:47:16.075108+00:00,2021-08-31 19:59:00-04:00


# Task description

The goal of this exercise is to implement the following transformations to the historical data:
- resampling
- VWAP, TWAP computation
- Calculation of returns

While using the different approaches to working with `dataflow` methods.
The main feature that these methods are trying to overcome is the fact that when the raw data consists of two and more `full_symbols`, then one needs to be careful to apply transformations that needs to be implemented specifically to each `full_symbol`.

These three approaches are:
- 1) Use the "low level" functions and do loops
- 2) Use pandas Multi-index
- 3) Use Dataflow nodes

The general rule is to use the third and second approach when possible, while keeping the first approach as a bacjup.

In [12]:
# The resampling frequency is the same for all approaches.
resampling_freq = config["data"]["resampling_rule"]

# Approach 1 - Use the "low level" functions and do loops

This approach does both resampling and computation of metrics and applied them individually to each `full_symbol` using the loop.

In [13]:
def resample_calculate_twap_vwap_and_returns(df, resampling_freq):
    result = []
    full_symbol_list = df["full_symbol"].unique()
    for cc in full_symbol_list:
        # DataFrame with a specific `full_symbol`
        cc_df = df[df["full_symbol"] == cc]
        # Resample OHLCV data inside `full_symbol`-specific DataFrame.
        resampled_cc_df = cfinresa.resample_ohlcv_bars(
            cc_df, rule=resampling_freq
        )
        # Attach VWAP, TWAP.
        resampled_cc_df[["vwap", "twap"]] = cfinresa.compute_twap_vwap(
            cc_df, resampling_freq, price_col="close", volume_col="volume"
        )
        # Calculate returns.
        resampled_cc_df["vwap_rets"] = cfinretu.compute_ret_0(
            resampled_cc_df[["vwap"]], "pct_change"
        )
        resampled_cc_df["twap_rets"] = cfinretu.compute_ret_0(
            resampled_cc_df[["twap"]], "pct_change"
        )
        resampled_cc_df["log_rets"] = cfinretu.compute_ret_0(
            resampled_cc_df[["close"]], "log_rets"
        )
        # Add a column with `full_symbol` indication.
        resampled_cc_df["full_symbol"] = cc
        # Omit unnecesary columns.
        resampled_cc_df = resampled_cc_df.drop(columns=["open", "high", "low"])
        result.append(resampled_cc_df)
    final_df = pd.concat(result)
    return final_df

In [14]:
df_approach_1 = resample_calculate_twap_vwap_and_returns(
    data_hist, resampling_freq
)
df_approach_1.head(3)

,close,volume,vwap,twap,vwap_rets,twap_rets,log_rets,full_symbol
end_ts,,,,,,,,
2021-08-31 20:00:00-04:00,47171.2,13.499,47171.200000,47171.20,NaN,NaN,NaN,binance::BTC_BUSD
2021-08-31 20:05:00-04:00,47117.9,213.698,47039.835039,47071.38,-0.002785,-0.002116,-0.001131,binance::BTC_BUSD
2021-08-31 20:10:00-04:00,47275.0,349.191,47305.716878,47295.82,0.005652,0.004768,0.003329,binance::BTC_BUSD


# Approach 2 - Use pandas Multi-index

In [15]:
# Drop non numerical columns to apply computations.
data_hist_num = data_hist.drop(
    columns=["full_symbol", "knowledge_timestamp", "start_ts"]
)

In [16]:
# Convert historical data to multiindex format.
converted_data = dtfcorutil.convert_to_multiindex(data_hist_num, "asset_id")
converted_data.head(3)

close                                                                                                                                                                                                                                                                               high                                                                                                                                                                                                                                                                                low                                                                                                                                                                                                                                                                               open                                                                                                                                      \
                          1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488   
end_ts                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
2021-08-31 20:00:00-04:00    47171.2    3430.49   47150.32     2.6312     13.073     31.359    0.72080      9.875    108.341     28.134     1.4750      2.235     1.1296      72.21     26.719     1.3383    0.27849        NaN    0.95392    3429.34      2.508     1.1872     39.545     464.44     5.2867    47205.7    3433.78   47186.11     2.6383     13.091     31.470    0.72161      9.895    108.565     28.185     1.4764      2.240     1.1319      72.29     26.756     1.3408    0.27860        NaN    0.95543    3432.51      2.513     1.1886     39.688     465.12     5.2929    47171.2    3430.13   47147.01     2.6312     13.064     31.357    0.72001      9.871    108.310     28.123     1.4710      2.235     1.1294      72.15     26.719     1.3377    0.27841        NaN    0.94999    3429.34      2.507     1.1870     39.488     464.43     5.2834    47192.7    3432.96   47184.11     2.6376     13.068     31.470    0.72143      9.879    108.513     28.171     1.4715      2.235     1.1319   
2021-08-31 20:01:00-04:00    47126.0    341

In [17]:
# Resampling VWAP (besides potential errors). This implies hardcoded formula in a mix with resampling functions.
vwap_approach_2 = (converted_data["close"] * converted_data["volume"]).resample(
    resampling_freq
).mean() / converted_data["volume"].resample(resampling_freq).sum()
vwap_approach_2.head(3)

,1182743717,1464553467,1467591036,1528092593,1776791608,1891737434,1966583502,2099673105,2237530510,2384892553,2425308589,2476706208,2484635488,2540896331,2601760471,2683705052,3065029174,3401245610,4516629366,4939988068,5115052901,5118394986,8717633868,8968126878,9872743573
end_ts,,,,,,,,,,,,,,,,,,,,,,,,,
2021-08-31 20:00:00-04:00,9405.744735,682.890751,9404.382989,0.524453,2.593709,6.207182,0.142914,1.961805,21.409336,5.567157,0.293490,0.444913,0.224603,14.385573,5.319561,0.266698,0.055410,NaN,0.188505,682.875259,0.500587,0.236079,7.846204,92.450821,1.049487
2021-08-31 20:05:00-04:00,9456.692974,687.878553,9453.114533,0.529330,2.602587,6.315404,0.144662,1.980988,21.790350,5.614976,0.295346,0.449093,0.226337,14.434024,5.371546,0.268658,0.055792,NaN,0.189383,687.467531,0.501621,0.238017,7.959054,93.077638,1.063564
2021-08-31 20:10:00-04:00,9435.538862,686.225286,9430.442370,0.526107,2.601904,6.370452,0.144988,1.971314,21.791179,5.574109,0.293429,0.449048,0.225254,14.359818,5.349444,0.267894,0.055537,NaN,0.188433,686.011202,0.498549,0.236749,7.975842,92.706603,1.065778


In [18]:
# Compute the ret_0 on all assets. You don't need a loop! But the data needs to be in the "right" format
# (the variable one wants to loop on needs to be the outermost in the levels, so one needs to do swaplevel).
rets_approach_2 = converted_data.swaplevel(axis=1).pct_change()
rets_approach_2.head(3)

,1182743717,1464553467,1467591036,1528092593,1776791608,1891737434,1966583502,2099673105,2237530510,2384892553,2425308589,2476706208,2484635488,2540896331,2601760471,2683705052,3065029174,3401245610,4516629366,4939988068,5115052901,5118394986,8717633868,8968126878,9872743573,1182743717,1464553467,1467591036,1528092593,1776791608,1891737434,1966583502,2099673105,2237530510,2384892553,2425308589,2476706208,2484635488,2540896331,2601760471,2683705052,3065029174,3401245610,4516629366,4939988068,5115052901,5118394986,8717633868,8968126878,9872743573,1182743717,1464553467,1467591036,1528092593,1776791608,1891737434,1966583502,2099673105,2237530510,2384892553,2425308589,2476706208,2484635488,2540896331,2601760471,2683705052,3065029174,3401245610,4516629366,4939988068,5115052901,5118394986,8717633868,8968126878,9872743573,1182743717,1464553467,1467591036,1528092593,1776791608,1891737434,1966583502,2099673105,2237530510,2384892553,2425308589,2476706208,2484635488,2540896331,2601760471,2683705052,3065029174,3401245610,4516629366,4939988068,5115052901,5118394986,8717633868,8968126878,9872743573,1182743717,1464553467,1467591036,1528092593,1776791608,1891737434,1966583502,2099673105,2237530510,2384892553,2425308589,2476706208,2484635488,2540896331,2601760471,2683705052,3065029174,3401245610,4516629366,4939988068,5115052901,5118394986,8717633868,8968126878,9872743573
,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,low,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,open,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume,volume
end_ts,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-08-31 20:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-31 20:01:00-04:00,-0.000958,-0.003373,-0.000984,-0.002888,-0.004513,-0.008164,-0.009545,-0.006987,-0.008381,-0.003554,-0.004000,-0.002237,-0.002125,-0.002354,-0.003481,-0.003213,-0.004058,NaN,-0.008250,-0.002936,-0.000399,-0.003369,-0.005336,-0.003165,-0.006526,-0.000729,-0.000955,-0.000266,-0.002388,-0.000993,-0.002923,-0.001012,-0.001920,-0.001419,-0.001171,0.009415,-0.001339,-0.001679,-0.000553,-0.001308,-0.001641,-0.000359,NaN,-0.001277,-0.000530,-0.000398,-0.000925,-0.003276,-0.001311,-0.001058,-0.000958,-0.003589,-0.000997,-0.002888,-0.003827,-0.010428,-0.010055,-0.006585,-0.010091,-0.003520,-0.001292,-0.002685,-0.002036,-0.001940,-0.003967,-0.003065,-0.004023,NaN,-0.005253,-0.003318,-0.000399,-0.003286,-0.004254,-0.003531,-0.006322,-0.000456,-0.000719,-0.000716,-0.002199,0.000230,-0.003591,-0.000776,-0.000607,-0.001511,-0.001633,0.002446,0.000447,-0.002032,0.000554,-0.000748,0.000000,-0.000431,NaN,0.004263,-0.000871,-0.000797,-0.000842,-0.001288,-0.001054,-0.000832,0.496185,4.906939,1.724382,0.317313,1.095496,2.201304,4.755429,3.167047,5.138713,0.186273,11.748394,1.815614,-0.332122,1.139061,2.640236,-0.282871,1.134819,NaN,0.499354,4.677257,1.819689,1.053180,0.024881,3.759917,5.605342
2021-08-31 20:02:00-04:00,0.000149,0.000369,0.000448,0.001944,0.001306,0

In [19]:
# To go back to a flat index representation.
rets_approach_2.columns = ["".join(str(col)) for col in rets_approach_2.columns]
rets_approach_2.head(3)

,"(1182743717, 'close')","(1464553467, 'close')","(1467591036, 'close')","(1528092593, 'close')","(1776791608, 'close')","(1891737434, 'close')","(1966583502, 'close')","(2099673105, 'close')","(2237530510, 'close')","(2384892553, 'close')","(2425308589, 'close')","(2476706208, 'close')","(2484635488, 'close')","(2540896331, 'close')","(2601760471, 'close')","(2683705052, 'close')","(3065029174, 'close')","(3401245610, 'close')","(4516629366, 'close')","(4939988068, 'close')","(5115052901, 'close')","(5118394986, 'close')","(8717633868, 'close')","(8968126878, 'close')","(9872743573, 'close')","(1182743717, 'high')","(1464553467, 'high')","(1467591036, 'high')","(1528092593, 'high')","(1776791608, 'high')","(1891737434, 'high')","(1966583502, 'high')","(2099673105, 'high')","(2237530510, 'high')","(2384892553, 'high')","(2425308589, 'high')","(2476706208, 'high')","(2484635488, 'high')","(2540896331, 'high')","(2601760471, 'high')","(2683705052, 'high')","(3065029174, 'high')","(3401245610, 'high')","(4516629366, 'high')","(4939988068, 'high')","(5115052901, 'high')","(5118394986, 'high')","(8717633868, 'high')","(8968126878, 'high')","(9872743573, 'high')","(1182743717, 'low')","(1464553467, 'low')","(1467591036, 'low')","(1528092593, 'low')","(1776791608, 'low')","(1891737434, 'low')","(1966583502, 'low')","(2099673105, 'low')","(2237530510, 'low')","(2384892553, 'low')","(2425308589, 'low')","(2476706208, 'low')","(2484635488, 'low')","(2540896331, 'low')","(2601760471, 'low')","(2683705052, 'low')","(3065029174, 'low')","(3401245610, 'low')","(4516629366, 'low')","(4939988068, 'low')","(5115052901, 'low')","(5118394986, 'low')","(8717633868, 'low')","(8968126878, 'low')","(9872743573, 'low')","(1182743717, 'open')","(1464553467, 'open')","(1467591036, 'open')","(1528092593, 'open')","(1776791608, 'open')","(1891737434, 'open')","(1966583502, 'open')","(2099673105, 'open')","(2237530510, 'open')","(2384892553, 'open')","(2425308589, 'open')","(2476706208, 'open')","(2484635488, 'open')","(2540896331, 'open')","(2601760471, 'open')","(2683705052, 'open')","(3065029174, 'open')","(3401245610, 'open')","(4516629366, 'open')","(4939988068, 'open')","(5115052901, 'open')","(5118394986, 'open')","(8717633868, 'open')","(8968126878, 'open')","(9872743573, 'open')","(1182743717, 'volume')","(1464553467, 'volume')","(1467591036, 'volume')","(1528092593, 'volume')","(1776791608, 'volume')","(1891737434, 'volume')","(1966583502, 'volume')","(2099673105, 'volume')","(2237530510, 'volume')","(2384892553, 'volume')","(2425308589, 'volume')","(2476706208, 'volume')","(2484635488, 'volume')","(2540896331, 'volume')","(2601760471, 'volume')","(2683705052, 'volume')","(3065029174, 'volume')","(3401245610, 'volume')","(4516629366, 'volume')","(4939988068, 'volume')","(5115052901, 'volume')","(5118394986, 'volume')","(8717633868, 'volume')","(8968126878, 'volume')","(9872743573, 'volume')"
end_ts,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-08-31 20:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-31 20:01:00-04:00,-0.000958,-0.003373,-0.000984,-0.002888,-0.004513,-0.008164,-0.009545,-0.006987,-0.008381,-0.003554,-0.004000,-0.002237,-0.002125,-0.002354,-0.003481,-0.003213,-0.004058,NaN,-0.008250,-0.002936,-0.000399,-0.003369,-0.005336,-0.003165,-0.006526,-0.000729,-0.000955,-0.000266,-0.002388,-0.000993,-0.002923,-0.001012,-0.001920,-0.001419,-0.001171,0.009415,-0.001339,-0.001679,-0.000553,-0.0013

# Approach 3 - Use Dataflow nodes

One node does resampling and VWAP, TWAP calculations, the other does returns.

In [20]:
# Configure the node to do the TWAP / VWAP resampling.
node_resampling_config = {
    "in_col_groups": [
        ("close",),
        ("volume",),
    ],
    "out_col_group": (),
    "transformer_kwargs": {
        "rule": resampling_freq,
        "resampling_groups": [
            ({"close": "close"}, "last", {}),
            (
                {
                    "close": "twap",
                },
                "mean",
                {},
            ),
            (
                {
                    "volume": "volume",
                },
                "sum",
                {"min_count": 1},
            ),
        ],
        "vwap_groups": [
            ("close", "volume", "vwap"),
        ],
    },
    "reindex_like_input": False,
    "join_output_with_input": False,
}
# Put the data in the DataFlow format (which is multi-index).
converted_data = dtfcorutil.convert_to_multiindex(data_hist, "asset_id")
# Create the node.
nid = "resample"
node = dtfcore.GroupedColDfToDfTransformer(
    nid,
    transformer_func=cofinanc.resample_bars,
    **node_resampling_config,
)
# Compute the node on the data.
vwap_twap = node.fit(converted_data)
# Save the result.
vwap_twap_approach_3 = vwap_twap["df_out"]
vwap_twap_approach_3.head(3)

close                                                                                                                                                                                                                                                                               twap                                                                                                                                                                                                                                                                             volume                                                                                                                                                                                                                                                                                    vwap                                                                                                                                 \
                          1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052  3065029174 3401245610 4516629366 4939988068 5115052901  5118394986 8717633868 8968126878 9872743573    1182743717   1464553467    1467591036 1528092593 1776791608 1891737434 1966583502 2099673105  2237530510 2384892553 2425308589 2476706208   
_index.tmp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
2021-08-31 20:00:00-04:00    47171.2    3430.49   47150.32     2.6312     13.073     31.359    0.72080      9.875    108.341     28.134     1.4750      2.235     1.1296      72.21     26.719     1.3383    0.27849        NaN    0.95392    3429.34      2.508     1.1872     39.545     464.44     5.2867   47171.20   3430.490  47150.320    2.63120    13.0730    31.3590   0.720800     9.8750   108.3410    28.1340    1.47500     2.2350    1.12960     72.210    26.7190    1.33830   0.278490        NaN   0.953920   3429.340     2.5080    1.18720    39.5450    464.440    5.28670     13.499    944.407    150.469    13233.0      737.2    82514.6   257524.0     8764.0    10410.0     2624.1    24447.0    15519.1    27511.0     3387.0    6977.54   388432.0   1547580.0        NaN   355061.0     54.043    12301.0    908681.3    11133.0     821.30     4343.0  47171.200000  3430.490000  47150.320000   2.631200  13.073000  31.359000   0.720800   9.875000  108.341000  28.134000   1.475000   2.235000   
2021-08-31 20:05:00-04:00    47117.9    342

In [21]:
# Configure the node to calculate the returns.
node_returns_config = {
    "in_col_groups": [
        ("close",),
        ("vwap",),
        ("twap",),
    ],
    "out_col_group": (),
    "transformer_kwargs": {
        "mode": "pct_change",
    },
    "col_mapping": {
        "close": "close.ret_0",
        "vwap": "vwap.ret_0",
        "twap": "twap.ret_0",
    },
}
# Create the node that computes ret_0.
nid = "ret0"
node = dtfcore.GroupedColDfToDfTransformer(
    nid,
    transformer_func=cofinanc.compute_ret_0,
    **node_returns_config,
)
# Compute the node on the data.
rets = node.fit(vwap_twap_approach_3)
# Save the result.
vwap_twap_rets_approach_3 = rets["df_out"]
vwap_twap_rets_approach_3.head(3)

close.ret_0                                                                                                                                                                                                                                                                         twap.ret_0                                                                                                                                                                                                                                                                         vwap.ret_0                                                                                                                                                                                                                                                                              close                                                                                                                                      \
                           1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 4939988068 5115052901 5118394986 8717633868 8968126878 9872743573 1182743717 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488   
_index.tmp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
2021-08-31 20:00:00-04:00         NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN    47171.2    3430.49   47150.32     2.6312     13.073     31.359    0.72080      9.875    108.341     28.134     1.4750      2.235     1.1296   
2021-08-31 20:05:00-04:00   -0.001